In [0]:
profile="huge"

In [0]:
qhost = !qhost | grep godel

In [0]:
controller_script = "/tmp/ipcontroller.sh"
with open(controller_script, "w") as o:
    o.write("ipcontroller --profile=%s\n" % profile)
!chmod +x $controller_script

In [0]:
!qdel ipcontroller
!qdel iengine
!rm ipengine*
!rm ipcontroller*

In [0]:
import time, tempfile, os
cmd = "qsub -o /tmp -e /tmp -cwd -V -N ipcontroller -q godel199@godel97 %s" % controller_script
print cmd
!$cmd
time.sleep(10)
temp_files = []
for elem in [x.split() for x in qhost]:
    host = elem[0]
    cpus = int(elem[2])
    if cpus > 8:
        tmp = tempfile.NamedTemporaryFile(delete=False)
        temp_files.append(tmp.name)
        tmp.write("pkill -9 ipengine\n")
        for i in xrange(cpus):
            tmp.write("ipengine --profile=%s&\nsleep 5\n" % profile)
        tmp.write("sleep 365d\n")
        tmp.close()
        q = "all.q"
        if host == "godel97" or host == "godel199":
            q = "godel199"
        cmd = "qsub -o /tmp -e /tmp -cwd -V -N ipengine -q %s@%s %s" % (q, host, tmp.name)
        print cmd
        !chmod +x {tmp.name}
        !$cmd

In [0]:
total_cpus = 0
for elem in [x.split() for x in qhost]:
    total_cpus += int(elem[2])
total_cpus

##Check size of the cluster

In [0]:
from datetime import datetime
from IPython.parallel import Client
rc = Client(profile=profile)

In [0]:
dview = rc[:]
print len(dview)

In [0]:
while (1):
    print datetime.now().time(), len(rc[:])
    time.sleep(10)

## Kill ipengines

In [0]:
exclude = {"godel21", "godel25"}
kill_file = "/home/cfriedline/ipython/kill_ipengine.sh"
with open(kill_file, "w") as o:
    o.write("%s\n" % "kill -9 `ps aux | grep '[I]Python.parallel.engine' | awk '{print $2}'`")
for host in qhost:
    host = host.split()[0].strip()
    if host not in exclude:
        x = !ssh {host} bash {kill_file}
        print host

In [0]:
exclude = {"godel21", "godel25"}
for host in qhost:
    host = host.split()[0].strip()
    if not host in exclude:
        x = !ssh {host} ps aux | grep samtools
        if len(x) > 0 and 'cfriedline' in x:
            !ssh {host} killall -9 sh
        print host, x